In [1]:
# import des libairies
from transformers import *
import torch
import pandas as pd 
import numpy as np

In [2]:
# import des données 
df = pd.read_csv("../data/silver/job_description.csv")
display(df.head())
print(df.shape)

,job,description
0,ABAP Developer,ABAP developers are in charge of designing and...
1,ASIC Design Engineer,An ASIC design engineer typically works within...
2,ASIC Engineer,ASIC (application-specific integrated circuit)...
3,ASP.NET Developer,An ASP.NET developer must have experience and ...
4,Actuarial Associate,Actuarial associates assist senior actuaries i...


(219, 2)


In [3]:
df.isnull().sum()

job             0
description    51
dtype: int64

### preprocessing

In [4]:
df = df[df['description'].notna()]
df.shape

(168, 2)

In [5]:
# Remplacer "Read less" à la fin de chaque description par une chaîne vide
df['description'] = df['description'].str.replace("Read less$", "", regex=True)
# Pour s'assurer que les espaces blancs à la fin sont également enlevés (s'il y en a)
df['description'] = df['description'].str.strip()

In [6]:
description_en = df["description"].tolist()
texts = description_en[:20]
texts

["ABAP developers are in charge of designing and developing complex enterprise-level applications for their organization. They are responsible for creating complex algorithms and data structures, and one of their main duties is performance monitoring and subsequently revising programs. ABAP developers assist in the translation of logical data design to physical database design, develop effective strategies for recovering applications in case of data loss, suggest changes for scalability issues, and promote quality control according to service level contracts. They also assist in project management activities and in the assessment and proper resolution of complex software bugs. Additionally, they implement system applications, follow up as needed in case of glitches after deploying to end users, conduct technical evaluation of code requirements, and propose detailed changes. ABAP developers also lead the review of business development case, as well as the revision of tools, standards, a

### Model

In [7]:
# import des packages
def get_translation_model_and_tokenizer(src_lang, dst_lang):
  """
  Given the source and destination languages, returns the appropriate model
  See the language codes here: https://developers.google.com/admin-sdk/directory/v1/languages
  For the 3-character language codes, you can google for the code!
  """
  # construct our model name
  model_name = f"Helsinki-NLP/opus-mt-{src}-{dst}"
  # initialize the tokenizer & model
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
  # return them for use
  return model, tokenizer

In [8]:
# source & destination languages
src = "en"
dst = "fr"

model, tokenizer = get_translation_model_and_tokenizer(src, dst)

In [9]:
def translate_texts(texts, model, tokenizer):
    """
    Translates a list of texts using the provided model and tokenizer.
    """
    # Tokenize the list of texts
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids

    # Generate the translation output using beam search
    beam_outputs = model.generate(inputs, num_beams=3, max_length=512)

    # Decode each translated output
    translations = [tokenizer.decode(output, skip_special_tokens=True) for output in beam_outputs]
    
    return translations

In [10]:
translated_texts = translate_texts(texts, model, tokenizer)

In [12]:
translated_texts[:3]

["Les concepteurs d'ABAP aident également à la gestion de projets et à l'évaluation et à la résolution appropriée des bogues logiciels complexes. Ils mettent en œuvre des applications système, assurent le suivi nécessaire en cas de problèmes d'embauche après avoir été déployés auprès des utilisateurs finaux, effectuent une évaluation technique des exigences de code et proposent des changements détaillés. Les concepteurs d'ABAP doivent également diriger l'examen de la situation de développement des affaires, ainsi que la révision des outils, des normes et des méthodologies. Ils créent des rapports à l'aide d'un ordinateur personnel, soutiennent la gestion des tâches d'embauche et fournissent un soutien régulier aux systèmes d'application et aux progiciels existants. Les concepteurs d'ABAP doivent suivre des mesures strictes de protection de la vie privée et des pratiques de sécurité du système.",
 "Un ingénieur de conception ASIC fonctionne généralement au sein d'une équipe qui est resp